In [2]:
import sqlite3
import requests
import pandas as pd
from datetime import datetime
import logging
import pytz

# Configuração de logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
def carregar_tabela_para_dataframe(nome_tabela, banco_de_dados):
    conn = sqlite3.connect(banco_de_dados)
    
    try:
        # Verificar se a tabela existe
        cursor = conn.cursor()
        cursor.execute("""
            SELECT name FROM sqlite_master WHERE type='table' AND name=?;
        """, (nome_tabela,))
        
        resultado = cursor.fetchone()

        if resultado:
            # Se a tabela existe, carregar seus dados para um DataFrame
            df = pd.read_sql_query(f"SELECT * FROM {nome_tabela}", conn)
            return df
        else:
            # Retornar um DataFrame vazio se a tabela não existir
            return pd.DataFrame()  
    finally:
        conn.close()  # Garantir que a conexão seja fechada
# Função para buscar e processar os dados de uma linha de ônibus
def buscar_e_processar_dados():
    try:
        # Definir o fuso horário
        timezone_sp = pytz.timezone('America/Sao_Paulo')
        hora_online = datetime.now(timezone_sp).strftime("%Y-%m-%d %H:%M:%S")
        
        # URL da página com os dados
        url = "https://transporteservico.urbs.curitiba.pr.gov.br/getOcorrenciaCCOporLinha.php?c=821f0"
        
        # Fazer a requisição GET
        response = requests.get(url, timeout=10)
        
        # Verificar se a requisição foi bem-sucedida
        if response.status_code == 200:
            dados = response.json()  # Converter o conteúdo para JSON
            
            # Criar um DataFrame com os dados
            if dados:  # Verificar se a resposta contém dados
                df_codigos_onibus = pd.DataFrame(dados)
                df_codigos_onibus['HORA'] = hora_online
                return df_codigos_onibus
            else:
                logging.warning("Nenhum dado foi retornado da URL.")
                return None

    except requests.Timeout:
        logging.error("Timeout: A conexão demorou mais de 10 segundos.")
    except requests.RequestException as e:
        logging.error(f"Erro na requisição: {e}")
    except Exception as e:
        logging.error(f"Erro ao processar os dados: {e}")
    return None

# Função para criar a tabela se ela não existir
def criar_tabela_se_nao_existir(conn):
    cursor = conn.cursor()
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS onibus (
            ID INTEGER,
            TIPO INTEGER,
            LINHA INTEGER,
            MENSAGEM TEXT,
            HORA TEXT,
            FLAG_PROCES INTEGER
        )
    ''')
    conn.commit()

# Função para gravar os dados no banco de dados SQLite
def gravar_no_banco(df_result):
    with sqlite3.connect('dados_ocorrencia_onibus.db') as conn:
        # Criar a tabela se não existir
        criar_tabela_se_nao_existir(conn)

        # Inserir dados no banco
        cursor = conn.cursor()
        for index, row in df_result.iterrows():
            cursor.execute('''
                INSERT INTO onibus (ID, TIPO, LINHA, MENSAGEM, HORA, FLAG_PROCES) 
                VALUES (?, ?, ?, ?, ?, ?)
            ''', (
                row['ID'], row['TIPO'], row['LINHA'], row['MENSAGEM'],
                row['HORA'], 0
            ))
        conn.commit()

# Função principal para executar o processo
def main():
    df_concatenado = buscar_e_processar_dados()
    if df_concatenado is not None:
        gravar_no_banco(df_concatenado)

# Iniciar a execução
if __name__ == "__main__":
    main()
